In [1]:
import json
from datetime import datetime, timedelta
import time
import paho.mqtt.client as mqtt

# ============================
# CONFIG
# ============================
BROKER = "broker.emqx.io"   # hoặc broker của bạn
PORT = 1883
TOPIC = "esp32/esp-01/attendance"

RFID_UID = "ADMIN"

START_DATE = datetime(2025, 12, 1, 8, 0)   # ngày giờ bắt đầu
HOURS_STEP = 2                             # cách nhau 2 tiếng
TOTAL_RECORDS = 100                        # số bản ghi cần gửi
SLEEP_BETWEEN_REQUESTS = 0.3               # delay giữa mỗi publish

# ============================
# MQTT CLIENT
# ============================

client = mqtt.Client()
client.connect(BROKER, PORT, 60)
client.loop_start()

print(f"Publishing to topic: {TOPIC}")

# ============================
# SEND MULTIPLE RECORDS
# ============================

current_time = START_DATE

for i in range(TOTAL_RECORDS):
    timestamp_str = current_time.isoformat() + "+07:00"  # format theo yêu cầu

    payload = {
        "rfid_uid": RFID_UID,
        "timestamp": timestamp_str
    }

    client.publish(TOPIC, json.dumps(payload))
    print(f"[{i+1}] Sent -> {payload}")

    # tăng thời gian lên 2 tiếng
    current_time += timedelta(hours=HOURS_STEP)

    # chờ 0.3s để tránh spam broker
    time.sleep(SLEEP_BETWEEN_REQUESTS)

client.loop_stop()
client.disconnect()

print("DONE!")


Publishing to topic: esp32/esp-01/attendance
[1] Sent -> {'rfid_uid': 'ADMIN', 'timestamp': '2025-12-01T08:00:00+07:00'}
[2] Sent -> {'rfid_uid': 'ADMIN', 'timestamp': '2025-12-01T10:00:00+07:00'}
[3] Sent -> {'rfid_uid': 'ADMIN', 'timestamp': '2025-12-01T12:00:00+07:00'}
[4] Sent -> {'rfid_uid': 'ADMIN', 'timestamp': '2025-12-01T14:00:00+07:00'}
[5] Sent -> {'rfid_uid': 'ADMIN', 'timestamp': '2025-12-01T16:00:00+07:00'}
[6] Sent -> {'rfid_uid': 'ADMIN', 'timestamp': '2025-12-01T18:00:00+07:00'}
[7] Sent -> {'rfid_uid': 'ADMIN', 'timestamp': '2025-12-01T20:00:00+07:00'}
[8] Sent -> {'rfid_uid': 'ADMIN', 'timestamp': '2025-12-01T22:00:00+07:00'}
[9] Sent -> {'rfid_uid': 'ADMIN', 'timestamp': '2025-12-02T00:00:00+07:00'}
[10] Sent -> {'rfid_uid': 'ADMIN', 'timestamp': '2025-12-02T02:00:00+07:00'}
[11] Sent -> {'rfid_uid': 'ADMIN', 'timestamp': '2025-12-02T04:00:00+07:00'}
[12] Sent -> {'rfid_uid': 'ADMIN', 'timestamp': '2025-12-02T06:00:00+07:00'}
[13] Sent -> {'rfid_uid': 'ADMIN', 'time

KeyboardInterrupt: 